# First test for the kaggle 

In [127]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import networkx as nx
import pandas as pd
from scipy import optimize
from scipy.linalg import cho_factor, cho_solve
from tqdm import tqdm
import scipy
import cvxopt
from cvxopt import matrix

The evaluation pages describes how submissions will be scored and how students should format their submissions. The metric is the AUC (area under curve). The data contains 2 classes. 
## Submission
Format Submission files should contain two columns: Id and Prediction. The file should contain a header and have the format described below. Id represents the identifier of the test example, ranging from 1 to 2000. The prediction is the corresponding logit which is a real number Ex: ``` Id,Prediction 1, -1.1 2, 3.2 3, -2.4 4,-0.5 5,2.1 6,0.1 7,-0.9 ``` Below, you will also find a piece of code for reading/writing the data. ```python import pickle as pkl import pandas as pd with open('training_data.pkl', 'rb') as file: train_graphs = pkl.load(file) with open('test_data.pkl', 'rb') as file: test_graphs = pkl.load(file) with open('training_labels.pkl', 'rb') as file: train_labels = pkl.load(file) # define your learning algorithm here # for instance, define an object called ``classifier'' # classifier.train(train_labels,train_graphs) # predict on the test data # for instance, test_preds = classifier.predict(test_graphs) Yte = {'Prediction' : test_preds} dataframe = pd.DataFrame(Yte) dataframe.index += 1 dataframe.to_csv('test_pred.csv',index_label='Id') ```

In [3]:
with open('data/training_data.pkl', 'rb') as file: train_graphs = pkl.load(file)
with open('data/test_data.pkl', 'rb') as file: test_graphs = pkl.load(file)
with open('data/training_labels.pkl', 'rb') as file: train_labels = pkl.load(file)

In [4]:
new_train_labels = 2*train_labels-1

In [5]:
all_labels_nodes = []
for i in range(len(train_graphs)):
    for j in range(len(train_graphs[i].nodes)):
        all_labels_nodes.append(train_graphs[i].nodes[j]['labels'][0])

In [6]:
np.unique(all_labels_nodes,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([15755, 68731,  6900,  1014,  1893,  1316,   277,    98,   293,
          106,    25,    36,   239,     1,     7,     8,    20,     1,
            6,    11,     3,     2,     3,     1,     3,     4,    38,
            1,     5,     9,     7,     3,     1,     1,     4,     1,
            4,     1,     3,     1,     1,     3,     2,     1,     1,
            1,     1]))

In [7]:
all_labels_edges = []
for i in range(len(train_graphs)):
    for e in train_graphs[i].edges:
        all_labels_edges.append(train_graphs[i].edges[e]['labels'][0])

In [8]:
np.unique(all_labels_edges,return_counts=True)

(array([0, 1, 2, 3]), array([60228,  8812, 28308,   238]))

## nth order walk kernel

In [92]:
class walk_kernel:
    def __init__(self, order):
        self.order = order
        
    def similarity(self,G1,G2):
        # Input : G1,G2 two graphs
        # Output : K(G1,G2)
        product_graph = nx.cartesian_product(G1,G2)
        A = nx.adjacency_matrix(product_graph)
        n = A.shape[0]
        return np.ones((1,n)).dot(scipy.sparse.csr_matrix.power(A,self.order).dot(np.ones((n,1))))[0,0]
    
    def kernel(self,X,Y):
        # Input : X vector of N graphs, Y vector of M graphs
        # Output : K similarity matrix between X and Y
        N = len(X)
        M = len(Y)
        K = np.zeros((N,M))
        for i in tqdm(range(N)):
            for j in range(i,M):
                res = self.similarity(train_graphs[i],train_graphs[j])
                K[i,j] = res
                K[j,i] = res
        return K

In [93]:
walk_k = walk_kernel(3)

In [94]:
#N = len(train_graphs)
N = 100
K = walk_k.kernel(train_graphs[:N],train_graphs[:N])

  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/1919052559.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 100/100 [00:38<00:00,  2.59it/s]


## Geometric Random Walk Kernel

We have that the Geometric Random Walk Kernel is defined as follows :
$$K^{\infty}(G_1,G_2) = e^T(I - \lambda A_{\times})^{-1}e$$
where $e$ is the all-ones vector, $A_{\times}$ is the adjacency matrix of the product graph $G_1 \times G_2$ and $\lambda > 0$. The geometric random walk kernel converges only if $\lambda < \frac{1}{\lambda_{\times}}$ where $\lambda_{\times}$ is the largest eigenvalue of $A_{\times}$. The computation of this kernel is $\mathcal{O}(n^6)$.

In [95]:
class geometric_random_kernel:
    def __init__(self, lmd):
        self.lmd = lmd
        
    def similarity(self,G1,G2):
        # Input : G1,G2 two graphs
        # Output : K(G1,G2)
        product_graph = nx.cartesian_product(G1,G2)
        A = nx.adjacency_matrix(product_graph)
        n = A.shape[0]
        return (np.ones((1,n))@np.linalg.inv(np.eye(n) - self.lmd*A)@np.ones((n,1)))[0,0]
    
    def kernel(self,X,Y):
        # Input : X vector of N graphs, Y vector of M graphs
        # Output : K similarity matrix between X and Y
        N = len(X)
        M = len(Y)
        K = np.zeros((N,M))
        for i in tqdm(range(N)):
            for j in range(i,M):
                res = self.similarity(train_graphs[i],train_graphs[j])
                K[i,j] = res
                K[j,i] = res
        return K

In [96]:
k = geometric_random_kernel(0.01)

In [97]:
k.kernel(train_graphs[:100],train_graphs[:100])

  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/2612502025.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
  3%|▎         | 3/100 [00:09<05:07,  3.17s/it]


KeyboardInterrupt: 

## Vertex histogram kernel

This kernel is based on slide 20 of [this presentation](https://www.ic.unicamp.br/~seminarios/palestras/2020/2020s1/2020-02-17.pdf). The vertex label histogram of a graph $G$ is a vector $f = [f_1,...,f_d]^T$ such that $f_i = |\{v \in V : l(v) = i\}|$ for each possible label $i$.
Then, we have $$k(G,G') = \langle f,f' \rangle$$
This is not a complet graph kernel.

In [44]:
class vertex_histogramm_kernel:
    def __init__(self, biggest_class):
        self.biggest_class = biggest_class
    
    def histogramm(self,G):
        labels_G = []
        for j in range(len(G.nodes)):
            labels_G.append(G.nodes[j]['labels'])
        labels_G = np.array(labels_G)

        f = np.array([len(labels_G[labels_G==i]) for i in range(self.biggest_class)])
        return f
    
    def similarity(self,G1,G2):
        f_1 = self.histogramm(G1)
        f_2 = self.histogramm(G2)
        return f_1@f_2
    
    def kernel(self,X,Y):
        # Input : X vector of N graphs, Y vector of M graphs
        # Output : K similarity matrix between X and Y
        N = len(X)
        M = len(Y)
        K = np.zeros((N,M))
        for i in tqdm(range(N)):
            for j in range(i,M):
                res = self.similarity(train_graphs[i],train_graphs[j])
                K[i,j] = res
                K[j,i] = res
        return K
        

In [107]:
k = vertex_histogramm_kernel(49)

In [48]:
k.kernel(train_graphs[:100],train_graphs[:100])

100%|██████████| 100/100 [00:00<00:00, 174.84it/s]


array([[ 174.,  263.,  175., ...,  277.,  472.,  643.],
       [ 263.,  410.,  267., ...,  420.,  720.,  980.],
       [ 175.,  267.,  178., ...,  277.,  472.,  643.],
       ...,
       [ 277.,  420.,  277., ...,  445.,  760., 1035.],
       [ 472.,  720.,  472., ...,  760., 1300., 1770.],
       [ 643.,  980.,  643., ..., 1035., 1770., 2410.]])

## Weisfeiler-Lehman kernel

In [138]:
nx.weisfeiler_lehman_graph_hash(train_graphs[0],iterations=1)

'9dad9b8af2d917c153b31dd18946bd83'

In [139]:
nx.weisfeiler_lehman_subgraph_hashes(train_graphs[0])

{0: ['bc6b88dc3f535192a81e9083e46bdafc',
  'a78f29cd5372131de03565b94594d8eb',
  'dd97e9a9b1ed3bb50df216e1112c659d'],
 1: ['bc6b88dc3f535192a81e9083e46bdafc',
  'a5357d9e80333772a3bfb85018871738',
  '11f8cb73d7f48a4130e496c3faf8b44b'],
 2: ['cd755ae4b4b31e985b6e3893b878bd2c',
  '7baf9cf9014bec34f17bfc7d80883fc3',
  '7f512307af7b2e5e201f7fbf214cf246'],
 3: ['793295b409f46a69bc4968008523d606',
  '859255ca748eaaf285e1bf79a2e3f85d',
  '83e831745ce920d06a209b95bbfa3ba0'],
 4: ['894dc79574bde41801cac14fd57ec400',
  'c9297775b8d1cf7f82e83e24e3bdea7d',
  'e9f63642d0d7e9a7bb9254f06145b299'],
 5: ['0ea5aaaa75acdcc10dfc8f72ac0d4373',
  'ea8f2db97e2d638016bfc33077563172',
  '93b7db244e2e280ade325d097be0d3dc'],
 6: ['793295b409f46a69bc4968008523d606',
  '8bc6d6870a6f494822dc2cdaa78ecb86',
  'bc28e3ef3030dc9e27a36489d40f5918'],
 7: ['793295b409f46a69bc4968008523d606',
  '859255ca748eaaf285e1bf79a2e3f85d',
  '2777099f800d4c55aa537da7687b648b'],
 8: ['793295b409f46a69bc4968008523d606',
  '859255ca748e

## Test of SVC

In [109]:
class KernelSVC:
    def __init__(self, C, kernel, epsilon = 1e-3):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon
        self.norm_f = None
        self.X = None
        self.y = None
    
    def fit(self, X, y, K):
       #### You might define here any variable needed for the rest of the code
        self.X = X
        self.y = y
        N = len(y)
        #K = kernel(X,X)
        M = np.diag(y)@K@np.diag(y)
        # Lagrange dual problem
        def loss(alpha):
            return  (1/2)*alpha.T@M@alpha - np.sum(alpha)#'''--------------dual loss ------------------ '''

        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return  M@alpha - np.ones_like(alpha)# '''----------------partial derivative of the dual loss wrt alpha -----------------'''

        # Constraints on alpha of the shape :
        # -  d - C*alpha  = 0
        # -  b - A*alpha >= 0
        fun_eq = lambda alpha: alpha.T@y # '''----------------function defining the equality constraint------------------'''        
        jac_eq = lambda alpha: y   #'''----------------jacobian wrt alpha of the  equality constraint------------------'''
        fun_ineq = lambda alpha: np.concatenate((C*np.ones_like(alpha) - alpha,alpha))  # '''---------------function defining the inequality constraint-------------------'''     
        jac_ineq = lambda alpha:  np.concatenate((-np.eye(N),np.eye(N))) # '''---------------jacobian wrt alpha of the  inequality constraint-------------------'''
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 
                        'fun': fun_ineq , 
                        'jac': jac_ineq})

        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints)
        self.alpha = optRes.x
        ## Assign the required attributes

        indices =  np.where((self.alpha>0) & (self.alpha<C))[0]
        self.margin_points = X[indices] #'''------------------- A matrix with each row corresponding to a point that falls on the margin ------------------'''
        self.b =  np.mean(y[indices] - self.separating_function(self.margin_points))#''' -----------------offset of the classifier------------------ '''
        self.norm_f = self.alpha.T@K@self.alpha # '''------------------------RKHS norm of the function f ------------------------------'''
        self.support = X[np.where(self.alpha>self.epsilon)[0]]
        
    ### Implementation of the separting function $f$ 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        return np.sum(np.diag(self.y*self.alpha)@kernel(self.X,x),axis=0)

    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2 * (d+self.b> 0) - 1

In [120]:
C = 10
N = 500
kernel = vertex_histogramm_kernel(49).kernel
X = train_graphs[:N]
y = new_train_labels[:N]
K = kernel(X,X)
model = KernelSVC(C=C, kernel=kernel)

100%|██████████| 500/500 [00:13<00:00, 35.89it/s] 


In [121]:
C = 1
model = KernelSVC(C=C, kernel=kernel)
model.fit(np.array(X), np.array(y), K)

/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/673968749.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(np.array(X), np.array(y), K)
100%|██████████| 500/500 [00:12<00:00, 38.57it/s] 


In [122]:
pred = model.predict(X)

100%|██████████| 500/500 [00:13<00:00, 36.50it/s] 


In [123]:
pred

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [124]:
y

array([-1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
        1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1,
       -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1,  1,
       -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
        1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [125]:
np.unique(y,return_counts=True)

(array([-1,  1]), array([441,  59]))

In [126]:
np.unique(pred,return_counts=True)

(array([-1]), array([500]))

In [58]:
model.alpha

array([ 2.88159433e+08,  4.01139584e+08,  5.38731638e+08,  1.93186559e+08,
        3.39479336e+09,  5.78086417e+08, -3.11367557e+08,  7.42625459e+08,
        6.70721048e+08,  2.39735659e+09,  8.92560729e+08,  3.20615012e+08,
       -2.82654618e+08, -1.85706411e+09, -3.10520268e+08, -3.11494206e+08,
       -3.73483736e+08,  3.05144311e+08, -3.64163821e+08, -2.36502952e+08,
       -2.06505347e+09,  9.48532920e+08, -1.53126626e+09, -9.58766064e+08,
        1.32354647e+09,  7.17429928e+08,  6.87866708e+08,  4.81810802e+08,
        9.08861401e+08, -2.64431990e+08, -1.48332668e+08,  7.76262555e+08,
        1.07483675e+09, -3.48985076e+08,  5.12024852e+08,  9.06891605e+08,
        2.34259313e+08, -1.27714862e+08,  6.46756240e+08, -2.16602341e+08,
       -5.58834480e+07, -8.78954314e+07,  1.02775454e+09,  2.67104344e+08,
        7.69685059e+08, -2.96993014e+08, -2.24142876e+08, -2.18548760e+08,
       -4.56709446e+08, -2.22377027e+08, -1.01988784e+09,  4.21345512e+09,
       -1.45374254e+09, -

## Logistic regression

In [103]:
def compute_correct(true_y,predicted_y):
    return np.sum(true_y == predicted_y)

In [12]:
def sigmoid(x):
    return 1/ (1 + np.exp(x))

In [76]:
class LogisticRegression():
    def __init__(self, lamb, kernel, eps = 1e-6, thresh = 0):
        """ 
        Parameters :
        lamb : float, regularization paramter
        eps : float, minimum tolerance
        thresh : float, minimum threshold for the prediction
        """
        self.lamb = lamb
        self.kernel = kernel
        self.eps = eps
        self.threshold = thresh
        self.X = None
        self.y = None
        self.alpha = None
    
    def fit(self, X, y, K, nb_iter = 10):
        """
        Parameters :
        X : array of training data (nb_samples x nb_features)
        y : list of training labels in {-1,1} (nb_samples)
        K : precomputed Gram matrix of X using kernel (nb_samples x nb_samples)
        
        Returns :
        alpha : nb_samples array
        """
        self.X = X
        self.y = y
        
        nb_samples = len(X)
        
        alpha = np.zeros(nb_samples)
        
        # Solve the KLR optimization problem by IRLS
        for i in range(nb_iter):
            new_alpha = alpha.copy()
            m = K@alpha
            W = sigmoid(m) * sigmoid(-m)
            z = m + y / sigmoid(-y * m)
            
            W_sqrt = np.sqrt(W)
            new_alpha = W_sqrt * np.linalg.solve(W_sqrt * K * W_sqrt.T + nb_samples * self.lamb * np.eye(nb_samples), W_sqrt * y)
            
            # If we have reached eps-close to the previous iterate, we stop
            if np.linalg.norm(alpha - new_alpha) < self.eps:
                alpha = new_alpha.copy()
                print("Stopped because convergence was reached at iteration", i)
                break
            
            alpha = new_alpha.copy()
        
        self.alpha = alpha
        #return alpha
    
    def predict_logit(self,X):
        """
        Parameters :
        X : array of data for which we want to predict the label (nb_samples x nb_features)
        
        Returns :
        Array of the predicted logits (n_samples)
        """
        K = self.kernel(X,self.X)
        return K@self.alpha
    
    def predict_class(self, X):
        """
        Parameters :
        X : array of data for which we want to predict the label (nb_samples x nb_features)
        
        Returns :
        Array of the predicted labels (n_samples)
        """
        K = self.kernel(X, self.X)
        y = np.dot(K, self.alpha)
        return np.where(y > self.threshold, 1, -1)

In [13]:
walk_kernel

In [99]:
N = 100
X = train_graphs[:N]
y = new_train_labels[:N]
K = k.kernel(X,X)

  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/2612502025.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 100/100 [04:27<00:00,  2.67s/it]


In [100]:
model = LogisticRegression(1, k.kernel)
model.fit(X, y, K, nb_iter = 1000)

In [101]:
pred = model.predict_class(X)

  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/2612502025.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 100/100 [05:10<00:00,  3.10s/it]


In [104]:
compute_correct(y,pred)

23

In [105]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [106]:
K

array([[ 266.68800223,  400.38814767,  300.36986129, ...,  384.42605396,
         633.38684377,  867.45937936],
       [ 400.38814767,  601.11741291,  450.95638718, ...,  577.15393721,
         950.92611964, 1302.34853248],
       [ 300.36986129,  450.95638718,  338.30607827, ...,  432.97925684,
         713.38162445,  977.01775068],
       ...,
       [ 384.42605396,  577.15393721,  432.97925684, ...,  554.14777   ,
         913.01599171, 1250.43045255],
       [ 633.38684377,  950.92611964,  713.38162445, ...,  913.01599171,
        1504.30049551, 2060.22527408],
       [ 867.45937936, 1302.34853248,  977.01775068, ..., 1250.43045255,
        2060.22527408, 2821.59802272]])

## SVM 

In [141]:
class SVM:
    def __init__(self, C, kernel, thresh=1e-3):
        """ 
        Parameters :
        C : float, regularization paramter
        thresh : float, minimum threshold for the prediction
        """
        self.C = C
        self.kernel = kernel
        self.threshold = thresh
        self.X = None
        self.y = None
        self.alpha = None
        self.support_vectors_indices = None
        self.support_vectors = None
        
    def fit(self, X, y, K, nb_iter = 10):
        """
        Parameters :
        X : array of training data (nb_samples x nb_features)
        y : list of training labels in {-1,1} (nb_samples)
        K : precomputed Gram matrix of X using kernel (nb_samples x nb_samples)
        
        Returns :
        alpha : nb_samples array
        """
        self.X = X
        self.y = y
        
        nb_samples = len(X)
        
        # We start by defining the different quantities we need in order to solve our problem

        P = matrix(K)
        q = matrix(-y.astype('float'))
        G = matrix(np.block([[np.diag(np.squeeze(y).astype('float'))],[-np.diag(np.squeeze(y).astype('float'))]]))
        h = matrix(np.concatenate((self.C*np.ones(nb_samples),np.zeros(nb_samples))))

        #Solve the problem using cvxopt
        solver = cvxopt.solvers.qp(P=P,q=q,G=G,h=h)
        solution = solver['x']
        self.alpha = np.squeeze(np.array(solution))

        #Retrieve the support vectors
        self.support_vectors_indices = np.squeeze(np.abs(np.array(solution))) > self.threshold
        self.alpha = self.alpha[self.support_vectors_indices]
        self.support_vectors = self.X[self.support_vectors_indices]
    
    def predict_logit(self,X):
        """
        Parameters :
        X : array of data for which we want to predict the label (nb_samples x nb_features)
        
        Returns :
        Array of the predicted logits (n_samples)
        """
        K = self.kernel(X,self.support_vectors)
        return K@self.alpha
    
    def predict_class(self, X):
        """
        Parameters :
        X : array of data for which we want to predict the label (nb_samples x nb_features)
        
        Returns :
        Array of the predicted labels (n_samples)
        """
        K = self.kernel(X, self.support_vectors)
        y = np.dot(K, self.alpha)
        return np.where(y > self.threshold, 1, -1)

In [171]:
N = 500
k = vertex_histogramm_kernel(49)
idx = np.random.choice(np.arange(len(train_graphs)),size = N, replace=False)
X = np.array(train_graphs)[idx]
y = new_train_labels[idx]
K = k.kernel(X,X)

/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_22938/4117056514.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(train_graphs)[idx]
100%|██████████| 500/500 [00:12<00:00, 38.72it/s] 


In [172]:
model = SVM(10, k.kernel)
model.fit(X, y, K)

     pcost       dcost       gap    pres   dres
 0: -1.3138e+03 -6.2305e+04  2e+05  1e+00  1e-11
 1: -1.0616e+03 -3.2130e+04  5e+04  2e-01  9e-12
 2: -9.6028e+02 -8.1225e+03  8e+03  3e-02  7e-12
 3: -1.0567e+03 -2.6288e+03  2e+03  4e-03  8e-12
 4: -1.1526e+03 -2.2973e+03  1e+03  3e-03  7e-12
 5: -1.2617e+03 -1.8662e+03  6e+02  1e-03  8e-12
 6: -1.3476e+03 -1.6257e+03  3e+02  5e-04  7e-12
 7: -1.4004e+03 -1.5157e+03  1e+02  2e-04  1e-11
 8: -1.4233e+03 -1.4707e+03  5e+01  6e-05  8e-12
 9: -1.4352e+03 -1.4499e+03  1e+01  1e-05  8e-12
10: -1.4387e+03 -1.4444e+03  6e+00  5e-06  9e-12
11: -1.4398e+03 -1.4425e+03  3e+00  2e-06  9e-12
12: -1.4408e+03 -1.4411e+03  3e-01  2e-16  1e-11
13: -1.4409e+03 -1.4409e+03  3e-03  2e-16  1e-11
14: -1.4409e+03 -1.4409e+03  3e-05  2e-16  9e-12
Optimal solution found.


In [173]:
pred = model.predict_class(X)

100%|██████████| 500/500 [00:01<00:00, 355.25it/s]


In [174]:
compute_correct(y,pred)

456

In [175]:
np.unique(pred,return_counts=True)

(array([-1]), array([500]))